In [1]:
# Import all packages here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import networkx as nx
from pyvis.network import Network
from collections import defaultdict
import glob
import os

In [2]:
df_device = pd.read_csv('../device_geo_data.csv')
df_network = pd.read_csv('../network_parent_child_resynced.csv')

In [3]:
cluster_results_file = pd.read_csv('Clustering Results/H070A_result.csv')

In [20]:
def most_common_ancestor(geo_id_list, roots, diagnostics = False):

    path = []
    
    for i in range(0, len(geo_id_list)):
        path.append(parent_path(geo_id_list[i]))
    if (diagnostics == True):
        print(path)
        print('\n')
    
    d = defaultdict(int)
    for i in path:
        for j in i:
            d[j]+=1
    if (diagnostics == True):
        print('Dictionary: ',d)
        print('\n')

    #printing root node of every path
    if (diagnostics == True):
        for k in path:
            roots.append(k[len(k)-1])
        print(roots)
        print('\n')        
    
    # Find item with Max Value in Dictionary
    itemMaxValue = max(d.items(), key=lambda x: x[1])
    if (diagnostics == True):
        print('Maximum Value in Dictionary : ', itemMaxValue[1])
        print('\n')
    
    listOfKeys = list()
    # Iterate over all the items in dictionary to find keys with max value
    for key, value in d.items():
        if value == itemMaxValue[1]:
            listOfKeys.append(key)
    if (diagnostics == True):
        print('Most common ancestors : ', listOfKeys)
        print('\n')

In [21]:
def parent_path(geo_id_entry, result = None):
    # create a new result if no intermediate was given
    if result is None:
        result = []
    try:
        parent = df_merged[df_merged['geo_id']==geo_id_entry]['parent_id'].iloc[0]
    except:
        return
    
    result.append(parent)
    parent_path(parent, result)
    
    return result

In [25]:
cwd_path = os.getcwd()
csv_files = glob.glob(os.path.join(cwd_path,'Clustering Results', "*.csv"))

for f in csv_files[0:41]:

    result_file = pd.read_csv(f)

    device_clusters = {}

    for i in range(len(set(result_file['y_pred']))):
        device_clusters[i] = result_file[result_file['y_pred'] == i]['device_id'].to_list()

    for counter, device_ids in device_clusters.items():

        device_data = df_device.loc[df_device['mac'].isin(device_ids)]

        # Joining the two dataframes for selected cluster
        df_merged = pd.merge(device_data, df_network, on='geo_id',how='left')
        df_merged = df_merged.dropna()

        df_merged['geo_id'].value_counts()

        # adds the frequency of occurence of a particular geo-id
        df_merged['freq'] = df_merged.groupby('geo_id')['geo_id'].transform('count')

        df_merged = df_merged.drop_duplicates(subset=['geo_id'])
        df_merged = df_merged.drop('mac',1)

        df_merged['parent_id'] = df_merged['parent_id'].astype(int)

        roots = []
        geo_id_list_input = df_merged['geo_id'].to_list()

        if geo_id_list_input:
            if (False):
                print("Cluster {}".format(counter+1))
                print('\n')
            most_common_ancestor(geo_id_list_input, roots, False) 
            G = nx.from_pandas_edgelist(df_merged, source= 'parent_id', target= 'geo_id', edge_attr= 'freq', 
                                    create_using=nx.DiGraph())

            net = Network('1000px', '1000px', directed = True, bgcolor='#222222', font_color='white')
            net.from_nx(G)

            result_path = cwd_path + '/Graph Results/' + f.split('/')[-1].split('.')[0] + '_cluster_' + str(counter) + '.html'
            net.save_graph(result_path)

/var/folders/fm/c1ww64d10qd453l8t8hrys300000gn/T/ipykernel_47195/1230877737.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_merged = df_merged.drop('mac',1)
